In [1]:
# RAG with Gemini Flash 1.5 LLM and ARES evaluation
# Google Gemini: https://ai.google.dev/gemini-api/docs/models/gemini
# ARES: https://github.com/stanford-futuredata/ARES

In [2]:
# Establish RAG pipeline with Gemini

In [1]:
import os
from IPython.display import display, Markdown
import pandas as pd

import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from llama_index.core import Document, VectorStoreIndex, Settings, StorageContext, load_index_from_storage
from llama_index.vector_stores.faiss import FaissVectorStore
#from llama_index.llms.gemini import Gemini
#from llama_index.embeddings.gemini import GeminiEmbedding

import faiss

In [2]:
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
# set up local API key
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

In [4]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash") # old function

In [5]:
# create document database
# using 4 State of the Union speeches, all text from whitehouse.gov briefing room speeches posted online, including a title with the date of the speech
# Example from 2024:
# https://www.whitehouse.gov/briefing-room/speeches-remarks/2024/03/07/remarks-of-president-joe-biden-state-of-the-union-address-as-prepared-for-delivery-2/
sotu = []
newfiles = ["./Speeches/titleedits/state_of_the_union_042921.txt", "./Speeches/titleedits/state_of_the_union_030122.txt", "./Speeches/titleedits/state_of_the_union_020723.txt", "./Speeches/titleedits/state_of_the_union_030724.txt"]
for i in newfiles:
    with open(i) as file:
        for line in file:
            nl = line.rstrip()
            if nl != '':
                sotu.append(nl)

In [6]:
documents = [Document(text=line) for line in sotu]

In [7]:
# Example of a loaded Document line
documents[-1]

Document(id_='02f5ebd8-b938-49aa-adfc-4dc668d30ad1', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='May God protect our troops.', mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n')

In [8]:
# Set up the faiss index
d = 768 # dimensions of the input vector of the embedding model that we're going to use; in this case, the google embedding model
faiss_index = faiss.IndexFlatL2(d)
print(faiss_index.is_trained)

True


In [9]:
#llm = Gemini(model="models/gemini-1.5-flash", api_key=os.environ["GOOGLE_API_KEY"])

In [10]:
# set up the embeddings
doc_embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004") # optional: task_type="RETRIEVAL_DOCUMENT"
#doc_embeddings = GeminiEmbedding(model="models/text-embedding-004")
Settings.embed_model = doc_embeddings
Settings.llm = llm

In [11]:
# test llm
#response = llm.complete("Write the text for an invitation for a two year old's penguin themed birthday party.")
#print(response)

In [13]:
## uncomment for when you need to re-embed and vectorize documents
## otherwise, doing local loading below
#vector_store = FaissVectorStore(faiss_index=faiss_index)
#storage_context = StorageContext.from_defaults(vector_store=vector_store)
#index = VectorStoreIndex.from_documents(
#    documents, storage_context=storage_context, show_progress=True
#)

In [14]:
# save index to disk
#index.storage_context.persist()
#index

In [11]:
# load index from disk
vector_store = FaissVectorStore.from_persist_dir("./storage")
storage_context = StorageContext.from_defaults(
    vector_store=vector_store, persist_dir="./storage"
)
# index id 'cef7ae30-ff1e-404a-bce6-85d59ca4b376' uses the speeches with a title that includes the date it was given
index = load_index_from_storage(storage_context=storage_context, index_id='cef7ae30-ff1e-404a-bce6-85d59ca4b376')

In [12]:
# set up query and chat engines
query_engine = index.as_query_engine(similarity_top_k=10)
chat_engine = index.as_chat_engine(similarity_top_k=10, chat_mode='context')

In [ ]:
# Example query and response
#query = "What does the President say about his administration's first 100 days and covid-19?"
#response = query_engine.query(query) 

In [15]:
print(response.response)

The President is proud of his administration's progress in fighting the pandemic, citing the successful rollout of COVID-19 vaccines. He highlights that they have surpassed their goal of administering 100 million vaccine shots in 100 days, reaching over 220 million shots. 



In [21]:
# Let's try to get ARES to work with Gemini and our local RAG setup

In [13]:
from ares import ARES

vLLM not imported.


In [14]:
ues_idp_config = {
    "in_domain_prompts_dataset": "ARES_files/nq_few_shot_prompt_for_judge_scoring.tsv",
    "unlabeled_evaluation_set": "ARES_files/nq_unlabeled_output.tsv", 
    "model_choice" : "models/gemini-1.5-flash",
    "request_delay" : 60,
    "documents" : 3
} 

ares = ARES(ues_idp=ues_idp_config)

In [18]:
results = ares.ues_idp()
print(results)

Evaluating large subset with models/gemini-1.5-flash:   0%|          | 0/3 [00:00<?, ?it/s]

configured gemini for context relevance
Testing response.text
[[Yes]] 

Testing candidates
['[[Yes]] \n']
configured gemini for answer relevance
Testing response.text
[[Yes]]

configured gemini for answer faithfulness
Testing response.text
[[Yes]]

configured gemini for context relevance
Testing response.text
[[Yes]]

Testing candidates
['[[Yes]]\n']
configured gemini for answer relevance
Testing response.text
[[Yes]]

configured gemini for answer faithfulness
Testing response.text
[[Yes]]

configured gemini for context relevance
Attempt 1 failed with error: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. Please check the `candidate.safety_ratings` to determine if the response was blocked.
Attempt 2 failed with error: Invalid operation: The `response.text` quick accessor requires the response to contain a valid `Part`, but none were returned. Please check the `candidate.safety_ratings` to determine if the re

In [18]:
results # exact scores repeated on second run

{'Context Relevance Scores': 0.667,
 'Answer Faithfulness Scores': 0.667,
 'Answer Relevance Scores': 0.667}

In [15]:
# stopped here: Need to generate synthetic question/answer/document file

# document refers to a line in a document_filepath_file; unsure if this setup of 1 large document per line will work
synth_config = { 
    "document_filepaths": ["Speeches/titleedits/Speeches_Docs6.tsv"], # requires tsv file...
    "few_shot_prompt_filename": "datasets/manual_dataset_complete_ares_synthetic.tsv",
    "synthetic_queries_filenames": ["results/synthetic_results.tsv"],
    "model_choice": "models/gemini-1.5-flash", # ex: "google/flan-t5-xxl" 
    "documents_sampled": 4, # was 10000
    "api_model": True
}

In [16]:
ares = ARES(synthetic_query_generator=synth_config)
results = ares.generate_synthetic_data() # stopped here, need to debug/edit to work with Gemini; next is embeddings error below
print(results)

Saving synthetic queries to:  ['results/synthetic_results.tsv']

| Starting Synthetic Query Generation |

Generating positive queries for the first 2 documents...


Generating positive synthetic queries for documents 0 to 2...:   0%|                              | 0/4 [00:00<?, ?it/s]

configured gemini for synthetic query generation
Testing response.text
What is the date of the State of the Union Address mentioned in the document? 

Testing response.text
What is the date of the State of the Union Address mentioned in the document? 

Testing response.text
What is the date of President Biden's State of the Union Address mentioned in the document? 

Testing response.text
What are the opening remarks of President Biden's 2024 State of the Union Address? 

Testing response.text
What is the date of the State of the Union Address mentioned in the document? 

Testing response.text
What is the date of the State of the Union Address mentioned in the document? 

Testing response.text
What is the date of the State of the Union Address mentioned in the document? 

Testing response.text
What is the main message that President Biden conveyed in his 2024 State of the Union address? 

Testing response.text
What is the date of the State of the Union Address mentioned in the document?

Generating positive synthetic queries for documents 0 to 2...:  25%|█████▌                | 1/4 [00:06<00:18,  6.33s/it]

Testing response.text
What was the context of the 1941 State of the Union address delivered by President Franklin Roosevelt? 

configured gemini for synthetic query generation
Testing response.text
What is the date of the State of the Union Address mentioned in the document? 

Testing response.text
What is the date of the State of the Union Address mentioned in the text? 

Testing response.text
What is the purpose of President Biden's 2024 State of the Union Address? 

Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).
Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).
Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).
Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).
Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).
Failed to generate synthetic queries after 5 attempts for percentile 0.95
Error genera

...

Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).
Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).
Failed to generate synthetic queries after 5 attempts for percentile 0.95
configured gemini for synthetic query generation
Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).
Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).
Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).
Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).
Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).
Failed to generate synthetic queries after 5 attempts for percentile 0.05
Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).
Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).
Error gener

Generating positive synthetic queries for documents 0 to 2...: 100%|██████████████████████| 4/4 [00:23<00:00,  5.98s/it]


Error generating synthetic queries: 429 Resource has been exhausted (e.g. check quota).
Failed to generate synthetic queries after 5 attempts for percentile 0.95
Total queries generated before filtering: 15
Total queries after length filtering: 15
Total queries after deduplication: 7


Generating embeddings...:  50%|████████████████████████████▌                            | 1/2 [00:00<00:00, 3057.07it/s]


OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [23]:
%tb

KeyError: 'Context_Relevance_Label'